In [111]:
import pandas as pd
import numpy as np
import scanpy
import scipy
from sklearn.decomposition import PCA
# from scipy.spatial.distance import cosine
# from sklearn.preprocessing import StandardScaler



In [116]:
# check arguments
# check that no columns or rows with all zeros
#ensure that factor variables are NOT integers or real
# what if no covariates

class condPCA(object):
    def __init__(self, count_matrix_path, metadata_path, object_columns, vars_to_regress=True):
        """
        Parameters
        ----------
        count_matrix:
            Count matrix that must be QC'd

        metadata:
            metadata containing cell type labels named "celltype"

        object_columns:
            columns that will be one hot encoded/columns that are factors 

        vars_to_regress:
            list of variables to regress out

        """
        self.count_matrix = scanpy.read(count_matrix_path) # cells x genes, pd.read_csv(count_matrix_path, sep='\t', header=0, index_col=0)
        self.metadata = pd.read_csv(metadata_path, sep='\t', header=0, index_col=0)
        if vars_to_regress:
            self.vars_to_regress = self.metadata.columns
        else: # if vars_to_regress is a list, convert to pandas core Index object
            self.vars_to_regress = pd.Index(vars_to_regress)

        # one hot encode necessary metadata variables
        self.object_columns = object_columns # obtain columns that must be one hot encoded
        self.metadata[self.object_columns] = self.metadata[self.object_columns].astype(object) # convert these columns to objects

    def Normalize(self):
        """ 
        Normalize and take log1p of count data
        """
        scanpy.pp.normalize_total(self.count_matrix, target_sum = 10000) # update scanpy object to normalize all rows, so every cell sums to 10k
        scanpy.pp.log1p(self.count_matrix) # log transform

    def Standardize(self):
        """ 
        Standardize count data AND metadata
        """
        # Standardize count data
        if scipy.sparse.issparse(self.count_matrix.X):
            self.count_matrix.X = self.count_matrix.X.todense()
        mean_count = np.mean(self.count_matrix.X, axis=0)
        std_count = np.std(self.count_matrix.X, axis=0)
        self.standardized_count_data = (self.count_matrix.X - mean_count) / std_count # standardize


        # Standardize metadata
        self.metadata = self.metadata[self.vars_to_regress] # subset to only variables that you want to regress out
        # WARNING IN FOLLOWING LINE BECAUSE CONVERTING OBJECT THAT LOOKS NUMERIC TO BE ONE HOT ENCODED, this is batch
        self.metadata = pd.get_dummies(self.metadata, drop_first=True) # Convert factor covariates to dummy variables dropping one column 
        mean = np.mean(self.metadata, axis=0) # compute mean of each column of metadata
        std = np.std(self.metadata, axis=0) # compute standard deviation of each column of metadata

        self.standardized_metadata = (self.metadata - mean) / std # standardize
        
    def _column_wise_regression(self, column): # perform regression of metadata/covariates across all genes
        beta = self.inv_cov @ self.standardized_metadata.T @ column
        return beta

    def fit(self):
        self.standardized_metadata = np.c_[np.ones((self.standardized_metadata.shape[0], 1)), self.standardized_metadata] # append ones to standardized meta for intercept
        self.inv_cov = np.linalg.pinv(np.matmul(self.standardized_metadata.T, self.standardized_metadata) ) # compute inverse of np.matmul(A^T, A) where A is the standardized metadata or covariates
        betas = np.apply_along_axis(self._column_wise_regression, axis=0, arr=self.standardized_count_data) # compute betas per gene
        prediction = np.matmul(self.standardized_metadata, betas) # compute prediction
        self.standardized_residual = self.standardized_count_data - prediction # compute residual
        # standardize residual:
        mean_resid = np.mean(self.standardized_residual, axis=0)
        std_resid = np.std(self.standardized_residual, axis=0)
        self.standardized_residual = (self.standardized_residual - mean_resid) / std_resid # standardize by gene
        # return standardized residual as a dataframe with gene and cell names:
        self.standardized_residual = pd.DataFrame(self.standardized_residual, index = list(self.count_matrix.obs_names), columns = list(self.count_matrix.var_names))

        # perform PCA
        pca = PCA(n_components=200) # instantiate PCA with hyperparameters
        self.cell_embeddings = pca.fit_transform(self.standardized_residual) # projections (of input data onto eigenvectors)
        self.gene_loadings = pca.components_ # retrieve eigenvectors
        

        


        
        

In [117]:
# instantiate class
test = condPCA(count_matrix_path="/Users/shayecarver/condPCA/final_method/test_matrix.txt", metadata_path="/Users/shayecarver/condPCA/final_method/test_metadata.txt", object_columns=['Batch', 'Sex','celltype'])
test.Normalize()
test.Standardize()
test.fit()

/var/folders/1b/d3mmyllj1jg4btvm4vqztkdh0000gp/T/ipykernel_1905/1976776286.py:57: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  self.metadata = pd.get_dummies(self.metadata, drop_first=True) # Convert factor covariates to dummy variables dropping one column


In [119]:
test.gene_loadings.shape
test.cell_embeddings.shape

(5000, 200)